In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df = pd.read_csv('data.csv', encoding='cp1252')


In [4]:
# Buat label klasifikasi
df['Profit_Class'] = df['Profit'].apply(lambda x: 'Untung' if x >= 0 else 'Rugi')

In [5]:
# Hapus kolom yang tidak dibutuhkan
df.drop(['Row ID', 'Order ID', 'Customer ID', 'Customer Name', 
         'Product ID', 'Product Name', 'Order Date', 'Ship Date', 'Postal Code'], axis=1, inplace=True)


In [6]:
# Cek dan hapus duplikat
df.drop_duplicates(inplace=True)

# Tangani missing values (jika ada)
df.dropna(inplace=True)  # atau imputasi jika diperlukan

In [7]:
# Tangani outlier pada kolom numerik
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

for col in ['Sales', 'Quantity', 'Discount', 'Profit']:
    df = remove_outliers_iqr(df, col)


In [10]:
# Pisahkan fitur dan target
X = df.drop('Profit_Class', axis=1)
y = df['Profit_Class']

# Identifikasi kolom
num_cols = ['Sales', 'Quantity', 'Discount']
cat_cols = X.select_dtypes(include='object').columns.tolist()


# Preprocessing
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_cols)  # 👈 ini yang diperbaiki
])


In [11]:
# Gabungkan pipeline
pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('pca', PCA(n_components=10))  # Sesuaikan jumlah komponen
])
X_transformed = pipeline.fit_transform(X)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)


In [13]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [14]:
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

        Rugi       0.72      0.44      0.55        77
      Untung       0.97      0.99      0.98      1269

    accuracy                           0.96      1346
   macro avg       0.85      0.72      0.76      1346
weighted avg       0.95      0.96      0.95      1346

Confusion Matrix:
 [[  34   43]
 [  13 1256]]


In [15]:
# Ambil 5 contoh data test dan prediksi
sample = X_test[:5]
sample_pred = model.predict(sample)

# Tampilkan
print("Prediksi:", sample_pred)
print("Label Asli:", y_test[:5].values)


Prediksi: ['Untung' 'Untung' 'Untung' 'Untung' 'Untung']
Label Asli: ['Rugi' 'Untung' 'Untung' 'Untung' 'Untung']
